In [12]:
!pip install moviepy
!pip install assemblyai
!pip install librosa
!pip install pydub
!pip install noisereduce
!pip install soundfile
!pip install huggingface_hub
!pip install transformer
!pip install evaluate
!pip install jiwer
!pip install sentencepiece
!pip install tensorflow_io
!pip install numpy scipy matplotlib scikit-learn simplejson eyed3 tqdm pyAudioAnalysis pyplot 
!pip install pyAudioAnalysis
!pip install hmmlearn
!pip install imblearn
!pip install eyeD3
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [13]:
from moviepy.editor import *
import librosa
import numpy as np
import matplotlib.pyplot as plt
import os
import assemblyai as aai
import pandas as pd
from pydub import AudioSegment
from pydub.silence import split_on_silence
import noisereduce as nr
import soundfile as sf
import tensorflow as tf
import torchaudio
from pyAudioAnalysis import audioSegmentation as aS
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [14]:
def video_to_audio(path_to_video):

  folder_path = path_to_video
  audio= []
  for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      if os.path.isfile(file_path) and filename.endswith(('.mp4', '.mov', '.avi', '.mkv')):
            print(f"Processing video file: {filename}")

            # Open the video file and extract audio
            videoclip = VideoFileClip(file_path)
            audioclip = videoclip.audio
            audio.append(audioclip)

            # Write the audio file to mp3 format
            audio_output_path = f"audios/audiofile_{filename.split('.')[0]}.mp3"
            audioclip.write_audiofile(audio_output_path, codec="libmp3lame")
            print(f"Audio saved as: {audio_output_path}")

  return audio

In [15]:
audio= video_to_audio('videos')

Processing video file: video_4CCXawD3Z1M.mp4
MoviePy - Writing audio in audios/audiofile_video_4CCXawD3Z1M.mp3


MoviePy - Done.
Audio saved as: audios/audiofile_video_4CCXawD3Z1M.mp3


In [20]:
import os
from datasets import Dataset, DatasetDict
import soundfile as sf

# def load_and_preprocess_data(audio_dir, transcript_dir):
def load_and_preprocess_data(audio_dir):
    data = {'audio': []}
    
    for index, audio_file in enumerate(os.listdir(audio_dir)):
        if audio_file.endswith('.mp3'):  # Adjust for audio file types
            file_path = os.path.join(audio_dir, audio_file)
 
            # Assuming preprocess_audio returns the processed audio and sample rate (sr)
#             processed_audio, sr = audio_reader(file_path)
            processed_audio, sample_rate = librosa.load(file_path, sr=16000, mono=True)

            # Append processed audio to the data dictionary
            data['audio'].append(processed_audio)

    return data

# Load and preprocess the audio and transcript data
# audio_data1 = load_and_preprocess_data('/content/sample_data/audios', '/content/sample_data/transcripts')
audio_data1 = load_and_preprocess_data("audios")



In [21]:

from pyAudioAnalysis import audioSegmentation as aS

def classify_audio(audio_file):
    # Perform mid-term file classification using the pre-trained SVM model for music/speech
    model_name = "data/svmRBF"  # Replace with actual path to the pre-trained model
    classifier_type = "svm"     # SVM classifier
    [flagsInd, classesAll, acc, CM] = aS.mid_term_file_classification(audio_file, model_name, classifier_type)
    return flagsInd  # 0 for speech, 1 for music

# Classify the given audio file
audio_category = classify_audio(audio_data1['audio'][0])

# Check result and output
if 1 in audio_category:
    print("The audio contains music.")
else:
    print("The audio contains speech.")



mtFileClassificationError: input model_type not found!
The audio contains speech.


In [7]:

def audio_2_speech(processor, model, audio_path):
    # Load your fine-tuned model and processor
    # processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    # model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    processor= processor
    model= model
    
    # Create the ASR pipeline with chunking enabled, explicitly providing feature_extractor and tokenizer
    asr_pipeline = pipeline(
        "automatic-speech-recognition",
        model=model,
        feature_extractor=processor.feature_extractor,  # Use the feature extractor
        tokenizer=processor.tokenizer,  # Use the tokenizer
        chunk_length_s=3  # Set chunk length in seconds (e.g., 5 seconds)
    )
    
    # Load the audio file path
    # audio_path = "/audios/audio_1.mp3"
    audio_path= audio_path
    
    # Perform inference with chunking and streaming
    transcription = asr_pipeline(audio_path)
    
    # Print the final transcription
    print("Transcription:", transcription['text'])

    return transcription

In [8]:
asr= audio_2_speech(Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h"), Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h"), audio_data1['audio'][0])

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Transcription: ON TELIVISION AND IN PRINT JOURNALISTS UNCOVER FACTS TO REPORT THE NEWS AS OBJECTIVELY AS POSSIBLE REPORTERS CORRESPONDENTS AND BROADCAST NEWS ANALISTS INFORMED THE PUBLIC ABOUT EVENTS AND NEWS OCCURRING INTERNATIONALLY NAIONALLY AND LOCALLY REPORTERS AND CORRESPONDENTS OR JOURNALISTS SPEND A LOT OF TIME IN THE FIELD MEETING CONTACTS INVESTIGATING STORIES AND CONDUCTING INTERVIEWS SO THEY CAN WRITE OR RECORD A STORY THE WORK IS OFTEN FAST PSTE TO MEET DEAD LINES OR BE THE FIRST TO BREAK NEWS MULTIMEDEAU SKILLS ARE INCREASINGLY IN DEMAND SO THAT JOURNALISTS CAN ADD AUDIO VIDIO AND GRAPHICS TO ADAPT STORIES FOR DIFFERENT PLATFORMS INCLUDING NEWSPAPERS MAGAZINES TELEVISION LIVE RADIO WEBSITES PODCASTES AND SOCIAL MEDIA SOME REPORTERS FREE LANCE COVERING INDIVIDUAL STORIES FOR A FEE OR MARKETING THEIR OWN STORIES TO NEWS ORGANIZATIONS BROADCAST NEWS ANNALYST'S WORK IN RADIO AND TELEVISION SHARING THEIR OPINIONS WITH THEIR AUDIENCE BASED ON THEIR EXPERTEESE IN A PARTICULAR SU

In [9]:
asr

{'text': "ON TELIVISION AND IN PRINT JOURNALISTS UNCOVER FACTS TO REPORT THE NEWS AS OBJECTIVELY AS POSSIBLE REPORTERS CORRESPONDENTS AND BROADCAST NEWS ANALISTS INFORMED THE PUBLIC ABOUT EVENTS AND NEWS OCCURRING INTERNATIONALLY NAIONALLY AND LOCALLY REPORTERS AND CORRESPONDENTS OR JOURNALISTS SPEND A LOT OF TIME IN THE FIELD MEETING CONTACTS INVESTIGATING STORIES AND CONDUCTING INTERVIEWS SO THEY CAN WRITE OR RECORD A STORY THE WORK IS OFTEN FAST PSTE TO MEET DEAD LINES OR BE THE FIRST TO BREAK NEWS MULTIMEDEAU SKILLS ARE INCREASINGLY IN DEMAND SO THAT JOURNALISTS CAN ADD AUDIO VIDIO AND GRAPHICS TO ADAPT STORIES FOR DIFFERENT PLATFORMS INCLUDING NEWSPAPERS MAGAZINES TELEVISION LIVE RADIO WEBSITES PODCASTES AND SOCIAL MEDIA SOME REPORTERS FREE LANCE COVERING INDIVIDUAL STORIES FOR A FEE OR MARKETING THEIR OWN STORIES TO NEWS ORGANIZATIONS BROADCAST NEWS ANNALYST'S WORK IN RADIO AND TELEVISION SHARING THEIR OPINIONS WITH THEIR AUDIENCE BASED ON THEIR EXPERTEESE IN A PARTICULAR SUBJECT

In [10]:

def summarization(tokenizer, model, transcription):
    
    # tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
    # model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
    tokenizer= tokenizer
    model= model
    
    text= transcription
    
    # Preprocessing the input text for T5
    input_text = "summarize: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    
    # Generate summary (set max_length to control the length of the summary)
    summary_ids = model.generate(input_ids, min_length= 150, max_length=500, num_beams=2, length_penalty=1.0)
    
    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    print("Summary:", summary)


In [11]:
summarization(AutoTokenizer.from_pretrained("facebook/bart-large-cnn"), AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn"), asr['text'])

Summary: JOURNALISTS UNCOVER FACTS TO REPORT THE NEWS. CORRESPONDENTS AND BROADCAST NEWS ANALISts INFORM THE PUBLIC ABOUT EVENTS AND NEWS OCCURRING. MULTIMEDEAU SKILLS are INCREASINGLY In DEMAND. JOURNalISTS CAN ADD AUDIO VIDIO AND GRAPHICS TO ADAPT STORIES. A BACHELOR'S DEGREE in JOURnalISM OR COMMUNICATIONS is PREFERRED. Journalists can add AUDIOVIDIO ANDGRAPHICS to ADAPt STORIES for DIFFERENT PLATFORMS. Journalists are required to report the news as OBJECTIVELY as possible.
